In [1]:
import os

os.chdir("../..")

import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [3]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train = X_train[:,:,0]
X_test = X_test[:,:,0]
X_val = X_val[:,:,0]


Caching the list of root modules, please wait!
Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)


(This will only be done once - type '%rehashx' to reset cache!)

This experiment has 43 classes


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D,MaxPool1D,AveragePooling1D
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import BatchNormalization, ReLU, MaxPooling1D, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate, TimeDistributed

from tensorflow.keras.models import Model, Sequential


In [7]:
inputs = Input(shape=(1000,1))
lstm_out = LSTM(64, return_sequences=True)(inputs)

query = Dense(64)(lstm_out)
value = Dense(64)(lstm_out)
att_out = Attention()([query, value])

concat = Concatenate()([lstm_out, att_out])
flatten = TimeDistributed(Dense(128))(concat)
x = Dense(64, activation='relu')(tf.keras.layers.Flatten()(flatten))
x = Dense(43, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=x)

In [11]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy',threshold=0.5),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(num_thresholds=200,
                                curve="ROC",
                                summation_method='interpolation',
                                name="AUC",
                                multi_label=True,
                                label_weights=None)]
)

In [12]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30)

Epoch 1/30
534/534 [==============================] - 743s 1s/step - loss: 0.1943 - accuracy: 0.9534 - recall: 0.3696 - AUC: 0.4968 - val_loss: 0.1210 - val_accuracy: 0.9610 - val_recall: 0.3488 - val_AUC: 0.4953
Epoch 2/30
534/534 [==============================] - 724s 1s/step - loss: 0.1366 - accuracy: 0.9598 - recall: 0.3735 - AUC: 0.5014 - val_loss: 3.0932 - val_accuracy: 0.8783 - val_recall: 0.6729 - val_AUC: 0.5042
Epoch 3/30
534/534 [==============================] - 741s 1s/step - loss: 0.2621 - accuracy: 0.9584 - recall: 0.3510 - AUC: 0.5119 - val_loss: 0.1235 - val_accuracy: 0.9610 - val_recall: 0.3488 - val_AUC: 0.5000
Epoch 4/30
534/534 [==============================] - 720s 1s/step - loss: 0.1217 - accuracy: 0.9609 - recall: 0.3488 - AUC: 0.4957 - val_loss: 0.1205 - val_accuracy: 0.9610 - val_recall: 0.3488 - val_AUC: 0.5000
Epoch 5/30
157/534 [=======>......................] - ETA: 8:13 - loss: 0.1203 - accuracy: 0.9607 - recall: 0.3469 - AUC: 0.4962

KeyboardInterrupt: 